In [7]:
import torch
from torch import nn, Tensor
import math
import torch.optim as optim

from s4 import S4Block as S4 
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
import random
import torch.nn.functional as F


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=500):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, feature_num]
        """
        x = x + self.pe[:x.size(1)].unsqueeze(0)
        return self.dropout(x)
    


class S4ModelForRUL(nn.Module):
    def __init__(self, d_input, d_model=512, n_layers=4, dropout=0.1, max_len=500):
        super().__init__()
        self.pos_encoder = PositionalEncoding(d_model, dropout=dropout, max_len=max_len)
        self.encoder = nn.Linear(d_input, d_model)
        self.bn_encoder = nn.BatchNorm1d(max_len) 
        self.s4_layers = nn.ModuleList()
        for _ in range(n_layers):
            self.s4_layers.append(S4(d_model, dropout=dropout, transposed=True))
        self.decoder = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
       
        src = self.encoder(src)  # [batch_size, seq_len, d_input] -> [batch_size, seq_len, d_model]
          # Apply tanh activation function
        src = self.bn_encoder(src)  
        src = F.tanh(src)
        #src = self.pos_encoder(src)
        src = src.transpose(1, 2)  # S4 expects [batch_size, d_model, seq_len]
        for layer in self.s4_layers:
            src, _ = layer(src)  # We ignore the state output here
        src = src.transpose(1, 2)  # Back to [batch_size, seq_len, d_model]
        src = self.dropout(src)
        output = self.decoder(src) 
        return output




In [8]:
def setup_optimizer(model, lr, weight_decay, epochs):
    """
    S4 requires a specific optimizer setup.

    The S4 layer (A, B, C, dt) parameters typically
    require a smaller learning rate (typically 0.001), with no weight decay.

    The rest of the model can be trained with a higher learning rate (e.g. 0.004, 0.01)
    and weight decay (if desired).
    """

    # All parameters in the model
    all_parameters = list(model.parameters())

    # General parameters don't contain the special _optim key
    params = [p for p in all_parameters if not hasattr(p, "_optim")]

    # Create an optimizer with the general parameters
    optimizer = optim.AdamW(params, lr=lr, weight_decay=weight_decay)

    # Add parameters with special hyperparameters
    hps = [getattr(p, "_optim") for p in all_parameters if hasattr(p, "_optim")]
    hps = [
        dict(s) for s in sorted(list(dict.fromkeys(frozenset(hp.items()) for hp in hps)))
    ]  # Unique dicts
    for hp in hps:
        params = [p for p in all_parameters if getattr(p, "_optim", None) == hp]
        optimizer.add_param_group(
            {"params": params, **hp}
        )

    # Create a lr scheduler
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=patience, factor=0.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

    # Print optimizer info
    keys = sorted(set([k for hp in hps for k in hp.keys()]))
    for i, g in enumerate(optimizer.param_groups):
        group_hps = {k: g.get(k, None) for k in keys}
        print(' | '.join([
            f"Optimizer group {i}",
            f"{len(g['params'])} tensors",
        ] + [f"{k} {v}" for k, v in group_hps.items()]))

    return optimizer, scheduler



In [9]:

class load_data(Dataset):
    """
    root = new | old
    """
    def __init__(self, name, seq_len, root='new') -> None:
        super().__init__()
        data_root = "data/units/"
        if root == 'old':
            label_root = "data/labels/"
        elif root == 'new':
            label_root = "data/new_labels/"
        else:
            raise RuntimeError("got invalid parameter root='{}'".format(root))
        raw = np.loadtxt(data_root+name)[:,2:]
        lbl = np.loadtxt(label_root+name)/Rc
        l = len(lbl)
        if l<seq_len:
            raise RuntimeError("seq_len {} is too big for file '{}' with length {}".format(seq_len, name, l))
        raw, lbl = torch.tensor(raw, dtype=torch.float), torch.tensor(lbl, dtype=torch.float)
        lbl_pad_0 = [torch.ones([seq_len-i-1]) for i in range(seq_len-1)] 
        data_pad_0 = [torch.zeros([seq_len-i-1,24]) for i in range(seq_len-1)]
        lbl_pad_1 = [torch.zeros([i+1]) for i in range(seq_len-1)] 
        data_pad_1 = [torch.zeros([i+1,24]) for i in range(seq_len-1)]
        self.data = [torch.cat([data_pad_0[i],raw[:i+1]],0) for i in range(seq_len-1)] 
        self.data += [raw[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
        self.data += [torch.cat([raw[l-seq_len+i+1:], data_pad_1[i]],0) for i in range(seq_len-1)]
        self.label = [torch.cat([lbl_pad_0[i],lbl[:i+1]],0) for i in range(seq_len-1)] 
        self.label += [lbl[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
        self.label += [torch.cat([lbl[l-seq_len+i+1:], lbl_pad_1[i]],0) for i in range(seq_len-1)]
        self.padding = [torch.cat([torch.ones(seq_len-i-1), torch.zeros(i+1)],0) for i in range(seq_len-1)]   # 1 for ingore
        self.padding += [torch.zeros(seq_len) for i in range(seq_len-1, l)]
        self.padding += [torch.cat([torch.zeros(seq_len-i-1), torch.ones(i+1)],0) for i in range(seq_len-1)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.label[index], self.padding[index]


class load_all_data(Dataset):
    """
    root: new | old
    name: LIST of txt files to collect 
    """
    def __init__(self, name, seq_len) -> None:
        super().__init__()
        data_root = "data/units/"
        label_root = "data/new_labels/"
        lis = os.listdir(data_root)
        data_list = [i for i in lis if i in name]
        self.data, self.label, self.padding = [], [], []
        for n in data_list:
            raw = np.loadtxt(data_root+n)[:,2:]
            lbl = np.loadtxt(label_root+n)/Rc
            l = len(lbl)
            if l<seq_len:
                raise RuntimeError("seq_len {} is too big for file '{}' with length {}".format(seq_len, n, l))
            raw, lbl = torch.tensor(raw, dtype=torch.float), torch.tensor(lbl, dtype=torch.float)
            lbl_pad_0 = [torch.ones([seq_len-i-1]) for i in range(seq_len-1)] 
            data_pad_0 = [torch.zeros([seq_len-i-1,24]) for i in range(seq_len-1)]
            lbl_pad_1 = [torch.zeros([i+1]) for i in range(seq_len-1)] 
            data_pad_1 = [torch.zeros([i+1,24]) for i in range(seq_len-1)]
            self.data += [torch.cat([data_pad_0[i],raw[:i+1]],0) for i in range(seq_len-1)] 
            self.data += [raw[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
            self.data += [torch.cat([raw[l-seq_len+i+1:], data_pad_1[i]],0) for i in range(seq_len-1)]
            self.label += [torch.cat([lbl_pad_0[i],lbl[:i+1]],0) for i in range(seq_len-1)] 
            self.label += [lbl[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
            self.label += [torch.cat([lbl[l-seq_len+i+1:], lbl_pad_1[i]],0) for i in range(seq_len-1)]
            self.padding += [torch.cat([torch.ones(seq_len-i-1), torch.zeros(i+1)],0) for i in range(seq_len-1)]   # 1 for ingore
            self.padding += [torch.zeros(seq_len) for i in range(seq_len-1, l)]
            self.padding += [torch.cat([torch.zeros(seq_len-i-1), torch.ones(i+1)],0) for i in range(seq_len-1)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.label[index], self.padding[index]

In [5]:
name = 'FD001'

In [12]:
tr = np.loadtxt("save/"+name+"/train"+name+".txt", dtype=str).tolist()
val = np.loadtxt("save/"+name+"/valid"+name+".txt", dtype=str).tolist()
ts = np.loadtxt("save/"+name+"/test"+name+".txt", dtype=str).tolist()

target = val


In [13]:
# add wandb
import wandb

wandb.login(key = '89972c25af0c49a4e2e1b8663778daedd960634a')

wandb.init(project="S4RUL", name="s4-rul")
#login to wandb

device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
d_input = 24  
seq_len = 70
Rc = 130
model = S4ModelForRUL(d_input=d_input, d_model=512, n_layers=1, dropout=0.1, max_len=seq_len)
model.to(device)
Loss = nn.MSELoss()
Loss.to(device)

# opt = torch.optim.Adam(model.parameters(), lr=0.01)
# sch = torch.optim.lr_scheduler.StepLR(opt, 50, 0.5)
epochs = 100
opt, sch = setup_optimizer(
    model, lr=0.02, weight_decay=1e-5, epochs=epochs
)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ValLoss,█▂▂▂▃▂▂▂▂▃▂▂▂▂▆▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
optimizer,████████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
scheduler,████████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
Epoch,99
Loss,0.00566
ValLoss,9.43151
optimizer,0.0
scheduler,0.0


Optimizer group 0 | 9 tensors | weight_decay 1e-05
Optimizer group 1 | 1 tensors | weight_decay 1e-05
Optimizer group 2 | 5 tensors | weight_decay 0.0


In [14]:
import torch
from torch.utils.data import DataLoader
import random
import math
import os

def get_score(pred, truth):
    """input must be tensors!"""
    x = pred-truth
    score1 = torch.tensor([torch.exp(-i/13)-1 for i in x if i<0])
    score2 = torch.tensor([torch.exp(i/10)-1 for i in x if i>=0])
    return int(torch.sum(score1)+torch.sum(score2))

def get_pred_result(data_len, out, lb):
    pred_sum, pred_cnt = torch.zeros(800), torch.zeros(800)
    for j in range(data_len):
        if j < seq_len-1:
            pred_sum[:j+1] += out[j, -(j+1):]
            pred_cnt[:j+1] += 1
        elif j <= data_len-seq_len:
            pred_sum[j-seq_len+1:j+1] += out[j]
            pred_cnt[j-seq_len+1:j+1] += 1
        else:
            pred_sum[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += out[j, :(data_len-j)]
            pred_cnt[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += 1
    truth = torch.tensor([lb[j,-1] for j in range(len(lb[0])-seq_len+1)], dtype=torch.float)
    
    pred_sum, pred_cnt = pred_sum[:data_len-seq_len+1], pred_cnt[:data_len-seq_len+1]
    pred2 = pred_sum/pred_cnt
    pred2 *= Rc
    truth *= Rc
    return truth, pred2 

def train(data, model, loss_function, optimizer, seq_len, epochs, device, name):
    min_rmse = float('inf')
    for e in range(epochs):
        model.train()
        random.shuffle(data)
        train_data = load_all_data(data, seq_len=seq_len)  # Ensure this returns a dataset compatible with DataLoader
        total_loss = 0.0
        
        train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
       
        for train_data, train_label, train_padding in train_loader:
            
            train_data, train_label = train_data.to(device), train_label.to(device)
            optimizer.zero_grad()
            output = model(train_data).squeeze()  # Adjusted to pass only train_data
            
            
            loss = loss_function(output, train_label)
            
            total_loss += loss.item()
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
            optimizer.step()
            
            

        
       
           
        rmse = validate(model, seq_len, device, target, score = False) 
        wandb.log({"Epoch": e, "Loss": total_loss / len(train_loader), "ValLoss": rmse, 'optimizer': optimizer.param_groups[0]['lr'], 'scheduler': sch.get_last_lr()[0]})
        print(f"Epoch: {e}, Loss: {total_loss / len(train_loader)}, Val loss: {rmse}")
            
        
        if rmse < min_rmse:
            min_rmse = rmse
            torch.save(model.state_dict(), f'save/s4_{name}.pth')
        
        sch.step()
    
    return min_rmse

            
            
            
        
def validate(model, seq_len, device, val_data, testing = False, score = True):
    model.eval()
    total_loss = 0.0
    total_score = 0.0
    with torch.no_grad():
        for i in val_data:
            pred_sum, pred_cnt = torch.zeros(800), torch.zeros(800)
            valid_data = load_data(i, seq_len)  # Ensure this returns a dataset compatible with DataLoader
            valid_loader = DataLoader(valid_data, batch_size=1000, shuffle=False)
            
           
            for valid_data, valid_label, valid_padding in valid_loader:
               
                valid_data = valid_data.to(device)
                data_len = len(valid_data)
                output = model(valid_data).squeeze(2).cpu()  # Adjusted to pass only valid_data
                # Proceed with your RMSE calculation

                
                
                for j in range(data_len):
                    if j < seq_len-1:
                    
                        pred_sum[:j+1] += output[j, -(j+1):]
                        pred_cnt[:j+1] += 1
                    elif j <= data_len-seq_len:
                        pred_sum[j-seq_len+1:j+1] += output[j]
                        pred_cnt[j-seq_len+1:j+1] += 1
                    else:
                        pred_sum[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += output[j, :(data_len-j)]
                        pred_cnt[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += 1
                truth = torch.tensor([valid_label[j,-1] for j in range(len(valid_label)-seq_len+1)], dtype=torch.float) * Rc
                pred_sum, pred_cnt = pred_sum[:data_len-seq_len+1], pred_cnt[:data_len-seq_len+1]
                pred = pred_sum/pred_cnt * Rc
                
                
                mse = float(torch.sum(torch.pow(pred-truth, 2)))
                rmse = math.sqrt(mse/data_len)
                if score:
                    sc = get_score(pred, truth)
                    total_score += sc
                    if testing:
                        print(f'RMSE for {i} is {rmse}, score is {sc}')
                        print('-'*20)
                        
                else:
                    if testing:
                    
                        print(f'RMSE for {i} is {rmse}')
                        print('-'*20)
                        
                total_loss += rmse
        if score:
            return total_score/len(val_data), total_loss/len(val_data)
        else:        
            return total_loss/len(val_data)
                
               

train(tr, model, Loss, opt, seq_len, epochs, device, name)

Epoch: 0, Loss: 6.796982077315349, Val loss: 13.255811116205543
Epoch: 1, Loss: 0.016908917355829396, Val loss: 14.015228992744243
Epoch: 2, Loss: 0.014301676888371239, Val loss: 10.913010650616386
Epoch: 3, Loss: 0.012068757762175959, Val loss: 10.530173100835952
Epoch: 4, Loss: 0.011379879257465536, Val loss: 13.712009359513047
Epoch: 5, Loss: 0.010862489709177532, Val loss: 14.371095176020935
Epoch: 6, Loss: 0.010716358646498742, Val loss: 11.13789976453378
Epoch: 7, Loss: 0.010219217089281694, Val loss: 9.976751508418205
Epoch: 8, Loss: 0.009963445941212814, Val loss: 11.453026950807157
Epoch: 9, Loss: 0.010084876762325497, Val loss: 12.797629781812121
Epoch: 10, Loss: 0.009569004426609623, Val loss: 10.58275069349566
Epoch: 11, Loss: 0.009135505835827742, Val loss: 11.48179375436669
Epoch: 12, Loss: 0.009264894635290713, Val loss: 10.0178961679792
Epoch: 13, Loss: 0.009009217255044024, Val loss: 9.826691575017662
Epoch: 14, Loss: 0.009131206750139795, Val loss: 11.4946812198553
Ep

8.925476283536366

In [15]:


def get_pred_result(data_len, out, lb):
    pred_sum, pred_cnt = torch.zeros(800), torch.zeros(800)
    for j in range(data_len):
        if j < seq_len-1:
            pred_sum[:j+1] += out[j, -(j+1):]
            pred_cnt[:j+1] += 1
        elif j <= data_len-seq_len:
            pred_sum[j-seq_len+1:j+1] += out[j]
            pred_cnt[j-seq_len+1:j+1] += 1
        else:
            pred_sum[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += out[j, :(data_len-j)]
            pred_cnt[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += 1
    truth = torch.tensor([lb[j,-1] for j in range(len(lb[0])-seq_len+1)], dtype=torch.float)
    
    pred_sum, pred_cnt = pred_sum[:data_len-seq_len+1], pred_cnt[:data_len-seq_len+1]
    pred2 = pred_sum/pred_cnt
    pred2 *= Rc
    truth *= Rc
    return truth, pred2 

In [16]:
validate(model, seq_len, device, ts, testing=True, score=True)

RMSE for FD001-22.txt is 17.279470053566552, score is 1096
--------------------
RMSE for FD001-23.txt is 9.477355712161026, score is 353
--------------------
RMSE for FD001-36.txt is 10.703425497807444, score is 432
--------------------
RMSE for FD001-8.txt is 8.268860810959149, score is 248
--------------------
RMSE for FD001-10.txt is 8.651956697973485, score is 382
--------------------
RMSE for FD001-4.txt is 9.802460084901405, score is 446
--------------------
RMSE for FD001-83.txt is 7.111830312611018, score is 281
--------------------
RMSE for FD001-86.txt is 15.243207255533807, score is 960
--------------------
RMSE for FD001-84.txt is 17.59441738351902, score is 1422
--------------------
RMSE for FD001-19.txt is 7.458868144735544, score is 194
--------------------
RMSE for FD001-2.txt is 5.932286297566237, score is 212
--------------------
RMSE for FD001-88.txt is 11.999848781922507, score is 423
--------------------
RMSE for FD001-56.txt is 15.865034897653993, score is 933
---

(601.6666666666666, 11.138928906308982)